In [505]:
import numpy as np
import numpy.linalg as LA
from math import *
N = 0
accuracy = [1e-3, 1e-5, 1e-9]
koeffs = [1, 100, 250]
startPoint = np.array([1, 1])

In [506]:
import pandas as pd
results = pd.DataFrame(columns=['title', 'koeff', *[f'EPS={i}' for i in accuracy]])

In [507]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09


In [508]:
def func(x, a):
  global N
  N += 1
  return x[0] * x[0] + a * x[1] * x[1]

def fD0(x, a):
  global N
  N += 1
  return 2 * x

def fD1(x, a):
  global N
  N += 1
  return 2 * a * x

def gradient(x, a):
  return np.array([fD0(x[0], a), fD1(x[1], a)])

In [509]:
def gradientDescent(startPoint, epsilon, koeff, A):

    point = np.array(startPoint)
    grad = gradient(point, koeff)
    

    while (LA.norm(grad) >= epsilon):
        alpha_k = - np.dot(grad, -grad)/np.dot(A @ -grad, -grad)
        point = point - alpha_k * grad

        grad = gradient(point, koeff)

    return (point, func(point, koeff))


In [510]:
for koeff in koeffs:
    N_EPS = []
    A = np.array([[2, 0], [0, 2*koeff]])
    for eps in accuracy:
        gradientDescent(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"gradientDescent", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1],"EPS=1e-09":N_EPS[2]}).to_frame().T], ignore_index=True)


In [511]:
print(gradientDescent(startPoint, 1, 1e-9, A))
print(N)

(array([2.22044605e-16, 9.99999999e-01]), 9.999999980000002e-10)
5


In [512]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,1,5,5,5
1,gradientDescent,100,13,17,25
2,gradientDescent,250,13,17,21


In [513]:
def conjugateGradientMethod(startPoint, epsilon, koeff, A):
    point = np.array(startPoint, dtype=np.int64)
    grad = gradient(point, koeff)
    p_k = -grad
    k = 0

    while (LA.norm(grad) >= epsilon):
        alpha_k = - np.dot(grad, p_k)/np.dot(A @ p_k, p_k)
        point = point + alpha_k * p_k
        Ngrad = gradient(point, koeff)

        if (k + 1 == point.size):
            betta_k=0
        else:
            betta_k = LA.norm(Ngrad)**2/LA.norm(grad)**2
        p_k = -Ngrad + np.dot(betta_k, p_k)

        grad = Ngrad

    return (point, func(point, koeff))


In [514]:
for koeff in koeffs:
    N_EPS = []
    A = np.array([[2, 0], [0, 2*koeff]], dtype=np.int64)
    for eps in accuracy:
        conjugateGradientMethod(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"conjugateGradientMethod", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1],"EPS=1e-09":N_EPS[2]}).to_frame().T], ignore_index=True)

In [515]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,1,5,5,5
1,gradientDescent,100,13,17,25
2,gradientDescent,250,13,17,21
3,conjugateGradientMethod,1,10,5,5
4,conjugateGradientMethod,100,7,7,7
5,conjugateGradientMethod,250,7,7,7


In [516]:
def newtonsMethod(startPoint, epsilon, koeff, A):
    point = np.array(startPoint, dtype=np.int64)
    grad = gradient(point, koeff)
    invA = LA.inv(A)

    while (LA.norm(grad) >= epsilon):
        point = point - (invA @ grad)
        grad = gradient(point, koeff)
    return (point, func(point, koeff))

In [517]:
for koeff in koeffs:
    N_EPS = []
    A = np.array([[2, 0], [0, 2*koeff]], dtype=np.int64)

    for eps in accuracy:
        newtonsMethod(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"newtonsMethod", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1],"EPS=1e-09":N_EPS[2]}).to_frame().T], ignore_index=True)

In [518]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,1,5,5,5
1,gradientDescent,100,13,17,25
2,gradientDescent,250,13,17,21
3,conjugateGradientMethod,1,10,5,5
4,conjugateGradientMethod,100,7,7,7
5,conjugateGradientMethod,250,7,7,7
6,newtonsMethod,1,5,5,5
7,newtonsMethod,100,5,5,5
8,newtonsMethod,250,5,5,5


In [519]:
def rSimplex(startPoint, ribLength, epsilon, koeff):

    simplex = buildSimplexC(startPoint, ribLength, lambda x: func(x, koeff))
    simplex = sort(simplex)
    k = 0
    while ribLength > epsilon or k < simplex.shape[0]:
        if k == simplex.shape[0]:
            k = 0
            ribLength = ribLength * 0.5
            simplex = buildSimplexD(simplex[-1], ribLength, lambda x: func(x, koeff))
            continue
        newApex = reflect(simplex, k, lambda x: func(x, koeff))
        if (newApex[-1] >= simplex[k][-1]):
            k += 1
            continue
        else:
            simplex[k] = newApex
            simplex = sort(simplex)
            k = 0
    return simplex[-1]


def sort(simplex):
    return simplex[simplex[:, -1].argsort()][::-1]


def reflect(simplex, k, func):
    coords = np.vstack((simplex[k+1:, :-1], simplex[:k, :-1]))
    center = np.sum(coords, axis=0)

    newCoord = 2 * center / coords.shape[0] - simplex[k, :-1]
    newValue = func(newCoord)

    newApex = np.hstack((newCoord, newValue))
    return newApex


def buildSimplexD(minApex, ribLength, func):
    numberOfDimensions = minApex[:-1].shape[0]
    apexPoints = np.array(minApex)

    for i in range(2, numberOfDimensions + 2):
        coords = np.array([])
        for j in range(1, numberOfDimensions + 1):
            if (i == j + 1):
                newCoord = minApex[j - 1] + (sqrt(numberOfDimensions + 1) - 1)/(numberOfDimensions * sqrt(2)) * ribLength
                coords = np.append(coords, newCoord)
            else:
                newCoord = minApex[j - 1] + (sqrt(numberOfDimensions + 1) + numberOfDimensions - 1)/(
                    numberOfDimensions * sqrt(2)) * ribLength
                coords = np.append(coords, newCoord)
        coords = np.append(coords, (func(coords)))
        apexPoints = np.vstack((apexPoints, coords))
    # print('New simplex is: \n', apexPoints)
    return apexPoints


def buildSimplexC(centerPoint, ribLength, func):
    '''
    Для реализации выбран второй способ построения симплекса через центральную точку

    Parameters
    ----------
    centerPoint (np.array): центр симплекса
    ribLength (int): длинна ребра симплекса
    func(vector): float

    Returns
    -------
    np.array(x, y, z, etc, val)
    '''
    numberOfDimensions = centerPoint.shape[0]
    apexPoints = None
    for i in range(1, numberOfDimensions + 2):
        # i это номер вершины симплекса
        coords = np.array([])
        for j in range(1, numberOfDimensions + 1):
            # j это номер координаты симплекса (x=1, y=2, z=3, ...)
            if j < i-1:
                newCoord = centerPoint[j - 1]
                coords = np.append(coords, newCoord)
            elif j == i-1:
                newCoord = centerPoint[j - 1] + sqrt(j / (2 * j + 2)) * ribLength
                coords = np.append(coords, newCoord)
            elif j > i-1:
                newCoord = centerPoint[j - 1] - sqrt(1 / (j*(2*j + 2))) * ribLength
                coords = np.append(coords, newCoord)
        coords = np.append(coords, (func(coords)))
        apexPoints = np.array(coords) if apexPoints is None else np.vstack(
            (apexPoints, coords))
    return apexPoints


In [520]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        rSimplex(startPoint, 0.5, eps,  koeff)
        N_EPS.append(N)
        N = 0
    results = pd.concat(
        [results, pd.Series(
            {"title": "rSimplex", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1],
             "EPS=1e-09": N_EPS[2]}).to_frame().T],
        ignore_index=True)


In [521]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,1,5,5,5
1,gradientDescent,100,13,17,25
2,gradientDescent,250,13,17,21
3,conjugateGradientMethod,1,10,5,5
4,conjugateGradientMethod,100,7,7,7
5,conjugateGradientMethod,250,7,7,7
6,newtonsMethod,1,5,5,5
7,newtonsMethod,100,5,5,5
8,newtonsMethod,250,5,5,5
9,rSimplex,1,87,149,245


In [522]:
a = np.array([1, 2])
b = np.array([3, 4])
LA.norm(a-b)

2.8284271247461903

In [523]:
def coordinateDescent(startPoint, epsilon, koeff):
  newPoint = oldPoint = startPoint
  newValue = oldValue = func(startPoint, koeff)
  dimensionNumber = startPoint.shape[0]

  j = 1
  while (True):
    oldPoint = newPoint
    oldValue = newValue

    while j <= dimensionNumber:
      newPoint = bitwiseSearch(newPoint, j, oldValue, 0.5, lambda x: func(x, koeff), epsilon)
      j += 1

    newValue = func(newPoint, koeff)
    if isSolved(oldPoint, newPoint, oldValue, newValue, epsilon, epsilon ):
      break
    j = 1
  return (newPoint, newValue)

def bitwiseSearch(point, j, value, step, func, eps):
  oldValue = value
  newPoint = point
  directionVector = np.array([0] * point.shape[0])
  directionVector[j - 1] = 1
  # Micro-optimization
  if func(newPoint + step * directionVector) > func(newPoint - step * directionVector):
    step *= -1
  #
  while abs(step) > eps:
    newPoint = newPoint + step * directionVector
    newValue = func(newPoint)
    if (newValue > oldValue):
      step = -step / 4
    oldValue = newValue
  return newPoint

def isSolved(oldPoint, newPoint, oldValue, newValue, eps1, eps2):
  return LA.norm(oldPoint - newPoint) < eps1 or abs(oldValue - newValue) < eps2

In [524]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        coordinateDescent(startPoint, eps,  koeff)
        N_EPS.append(N)
        N = 0
    results = pd.concat(
        [results, pd.Series(
            {"title": "coordinateDescent", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1],
             "EPS=1e-09": N_EPS[2]}).to_frame().T],
        ignore_index=True)

In [525]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,1,5,5,5
1,gradientDescent,100,13,17,25
2,gradientDescent,250,13,17,21
3,conjugateGradientMethod,1,10,5,5
4,conjugateGradientMethod,100,7,7,7
5,conjugateGradientMethod,250,7,7,7
6,newtonsMethod,1,5,5,5
7,newtonsMethod,100,5,5,5
8,newtonsMethod,250,5,5,5
9,rSimplex,1,87,149,245


In [526]:
def HookeJeeves(startPoint, eps,  koeff):
  incrementVector = np.array([1] * startPoint.shape[0])
  oldPoint = startPoint
  decrementKoeff = 2
  while True:
    prePoint = preResearch(oldPoint, incrementVector, lambda x: func(x, koeff))
    if (oldPoint == prePoint).all():

      if isSolved(incrementVector, eps):
        break

      incrementVector = incrementVector / decrementKoeff
      continue
    oldPoint = oldPoint + (prePoint - oldPoint)
  return (oldPoint, func(oldPoint, koeff))

def isSolved(vector, eps):
  return LA.norm(vector) < eps

def preResearch(point, incrementVector, func):
  j = 1
  dimensionNumber = point.shape[0]
  baseVector = np.array([0] * dimensionNumber)
  oldPoint = point
  oldValue = func(oldPoint)

  while j <= dimensionNumber:
    baseVector[j - 1] = 1
    newPoint = point - incrementVector[j-1]*baseVector
    newValue = func(newPoint)
    if oldValue > newValue:
      oldPoint = newPoint
      oldValue = newValue
      baseVector[j - 1] = 0
      j += 1
      continue
    
    newPoint = point + incrementVector[j-1]*baseVector
    newValue = func(newPoint)
    if oldValue > newValue:
      oldPoint = newPoint
      oldValue = newValue
      baseVector[j - 1] = 0
      j += 1
      continue
    baseVector[j - 1] = 0
    j += 1
    
  return oldPoint

In [527]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        HookeJeeves(startPoint, eps,  koeff)
        N_EPS.append(N)
        N = 0
    results = pd.concat(
        [results, pd.Series(
            {"title": "HookeJeeves", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1],
             "EPS=1e-09": N_EPS[2]}).to_frame().T],
        ignore_index=True)

In [528]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,1,5,5,5
1,gradientDescent,100,13,17,25
2,gradientDescent,250,13,17,21
3,conjugateGradientMethod,1,10,5,5
4,conjugateGradientMethod,100,7,7,7
5,conjugateGradientMethod,250,7,7,7
6,newtonsMethod,1,5,5,5
7,newtonsMethod,100,5,5,5
8,newtonsMethod,250,5,5,5
9,rSimplex,1,87,149,245


In [529]:
def randomSearch(startPoint, eps,  koeff):
  step = 2
  M = startPoint.shape[0] * 10
  newPoint = oldPoint = startPoint
  oldValue = func(startPoint, koeff)
  gamma = 2
  while step > eps:
    counter = 1
    while counter < M:
      randomVector = np.array([np.random.uniform(low=-1, high=1), np.random.uniform(low=-1, high=1)])
      newPoint = oldPoint + randomVector * step / LA.norm(randomVector)
      newValue = func(newPoint, koeff)
      if newValue >= oldValue:
        counter += 1
        continue
      while (newValue < oldValue):
        newPoint = newPoint + randomVector * step / LA.norm(randomVector)
        newValue = func(newPoint, koeff)
        oldValue = newValue
        oldPoint = newPoint
      break
    step /= gamma
  return (oldPoint, oldValue)

In [530]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        randomSearch(startPoint, eps,  koeff)
        N_EPS.append(N)
        N = 0
        print('Koeff:', koeff, 'Eps:', eps)
    results = pd.concat(
        [results, pd.Series(
            {"title": "randomSearch", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1],
             "EPS=1e-09": N_EPS[2]}).to_frame().T],
        ignore_index=True)

Koeff: 1 Eps: 0.001
Koeff: 1 Eps: 1e-05
Koeff: 1 Eps: 1e-09
Koeff: 100 Eps: 0.001
Koeff: 100 Eps: 1e-05
Koeff: 100 Eps: 1e-09
Koeff: 250 Eps: 0.001
Koeff: 250 Eps: 1e-05
Koeff: 250 Eps: 1e-09


In [531]:
results

,title,koeff,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,1,5,5,5
1,gradientDescent,100,13,17,25
2,gradientDescent,250,13,17,21
3,conjugateGradientMethod,1,10,5,5
4,conjugateGradientMethod,100,7,7,7
5,conjugateGradientMethod,250,7,7,7
6,newtonsMethod,1,5,5,5
7,newtonsMethod,100,5,5,5
8,newtonsMethod,250,5,5,5
9,rSimplex,1,87,149,245
